In [2]:
# Spine Classification Training Notebook
# This notebook demonstrates how to train a deep learning model for classifying spine images into two categories: **NormalFinal** and **ScolFinal**.

# 1. Setting up 

import os
import random
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# ---------------------------
# Parameters
target_height = 224
target_width = 224
batch_size = 16
val_split = 0.1
random_seed = 123

# Set seeds for reproducibility
random.seed(random_seed)
tf.random.set_seed(random_seed)

print(f"Target resolution: {target_height}x{target_width}")

2025-03-08 22:54:46.446849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Target resolution: 224x224


In [3]:
# 2. Data Loading

# Base directory containing the training images
base_dir = "Spines/CombinedTrainImages"
classes = ["NormalFinal", "ScolFinal"]

def get_file_list(class_dir):
    folder = os.path.join(base_dir, class_dir)
    return [os.path.join(folder, f) for f in os.listdir(folder) 
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]

# Create a dictionary with file lists for each class
files_dict = {c: get_file_list(c) for c in classes}


In [4]:
# 3. Creating Training and Validation Datasets

#In this section, we split the data into training and validation sets. For each class, we shuffle the file list and apply a fixed validation split.

train_files, train_labels = [], []
val_files, val_labels = [], []

for class_idx, class_name in enumerate(classes):
    files = files_dict[class_name]
    random.shuffle(files)
    split_idx = int(len(files) * val_split)
    
    # Validation data
    val_files.extend(files[:split_idx])
    val_labels.extend([class_idx] * split_idx)
    # Training data
    train_files.extend(files[split_idx:])
    train_labels.extend([class_idx] * (len(files) - split_idx))


In [5]:
# 4. Data Augmentation and Preprocessing

# This section defines functions for data augmentation and preprocessing:
# augment_image: Applies random flips, brightness/contrast adjustments, and random cropping.
# preprocess: Reads, decodes, resizes the image, and applies MobileNetV2 preprocessing.

def augment_image(image, label):
    # Apply random flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    # Apply random brightness and contrast adjustments
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    
    # Random crop to 200x200 and resize back to target dimensions
    image = tf.image.random_crop(image, size=[200, 200, 3])
    image = tf.image.resize(image, [target_height, target_width])
    
    return image, label

def preprocess(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [target_height, target_width])
    image = preprocess_input(image)
    return image, label


In [6]:
# 5. Build TensorFlow Datasets

#We create TensorFlow datasets for both training and validation. The training dataset is shuffled and augmented, while the validation dataset is only preprocessed.#

train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
train_ds = train_ds.shuffle(1000).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
val_ds = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [7]:
# 6. Building and Training the Model

#We use MobileNetV2 as our base model (initialized with ImageNet weights). The training is done in two phases:
#1. Phase 1 (Frozen Base): Only train the new classifier layers.
#2. Phase 2 (Fine-Tuning): Unfreeze the last 50 layers of the base model and fine-tune with a lower learning rate.

# Load the MobileNetV2 model without the top classifier layers
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(target_height, target_width, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# ---------------------------
# Phase 1: Train with Frozen Base
base_model.trainable = False
model = keras.Sequential([
    base_model,
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

# Compile the model with a higher learning rate for initial convergence
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("\n--- Training Phase 1 (Frozen Base) ---")
history_phase1 = model.fit(train_ds, epochs=20, validation_data=val_ds, verbose=1)

# ---------------------------
# Phase 2: Fine-Tuning
# Unfreeze the last 50 layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Compile with a lower learning rate for fine-tuning
model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("\n--- Training Phase 2 (Fine-Tuning) ---")
history_phase2 = model.fit(train_ds, epochs=10, validation_data=val_ds, verbose=1)



--- Training Phase 1 (Frozen Base) ---
Epoch 1/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 987ms/step - accuracy: 0.6760 - loss: 0.5924 - val_accuracy: 0.7143 - val_loss: 0.7274
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 907ms/step - accuracy: 0.8688 - loss: 0.2929 - val_accuracy: 0.7857 - val_loss: 0.6003
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 793ms/step - accuracy: 0.9220 - loss: 0.1747 - val_accuracy: 0.7857 - val_loss: 0.5905
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 820ms/step - accuracy: 0.9368 - loss: 0.1733 - val_accuracy: 0.8214 - val_loss: 0.5888
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 796ms/step - accuracy: 0.9545 - loss: 0.1558 - val_accuracy: 0.8214 - val_loss: 0.5920
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9465 - loss: 0.1592 - val_accuracy: 0.8929 - val_loss: 0.4665
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 894ms/step - accuracy: 0.9379 - loss: 0.1447 - val_accuracy: 0.8214 - val_loss: 0.6050
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 827ms/step - accur

In [8]:
# 7. Model Evaluation on Validation Set

# After training, we evaluate the model using the validation dataset by generating a confusion matrix and classification report.
# Save the trained model
model.save('spine_model.h5')

# Predict on validation dataset
y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_pred = np.argmax(model.predict(val_ds), axis=1)

print("\nValidation Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("\nValidation Classification Report:")
print(classification_report(y_true, y_pred, target_names=classes))


2025-03-08 23:08:57.726708: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step

Validation Confusion Matrix:
[[11  3]
 [ 0 14]]

Validation Classification Report:
              precision    recall  f1-score   support

 NormalFinal       1.00      0.79      0.88        14
   ScolFinal       0.82      1.00      0.90        14

    accuracy                           0.89        28
   macro avg       0.91      0.89      0.89        28
weighted avg       0.91      0.89      0.89        28



In [ ]:
# Conclusion

# In this training notebook, we:
# Loaded and preprocessed the dataset with augmentation.
# Built and trained a model using MobileNetV2 with a two-phase training approach.
# Evaluated the model using detailed metrics on the validation set.

#The final model (`spine_model.h5`) is now saved and ready for further testing or deployment.


# In this training notebook, we:
## - Loaded and preprocessed the dataset with augmentation.
## - Built and trained a model using MobileNetV2 with a two-phase training approach.
## - Evaluated the model using detailed metrics on the validation set.

# - The final model (`spine_model.h5`) is now saved and ready for further testing or deployment.
